In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import caffe
import os
#from collections import Counter

caffe.set_device(0)
caffe.set_mode_cpu()

In [14]:
def load_model_net(model_used):
    # Set the right path to your model definition file, pretrained model weights,
    # and the image you would like to classify.
    MODEL_FILE = '/home/gonzalo/models/VGG_FACE_deploy.prototxt'
    PRETRAINED = '/media/gonzalo/OS/mydata/weights_solvers/TO USE/'+model_used
    
    # load the model
    caffe.set_mode_gpu()
    caffe.set_device(0)
    net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                           mean=np.load('/home/gonzalo/data/MIT/mean_files/7_net_mean.npy').mean(1).mean(1),
                           channel_swap=(2,1,0),
                           raw_scale=255,
                           image_dims=(224, 224))
    print "successfully loaded classifier"
    return net

In [3]:
def prediction_net(files, test_path, angle_list, net):
    
    #reset variables
    tp, tp_0, tp_4, tp_8, tp_12, tp_16, tp_20, tp_24 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    fp, fp_0, fp_4, fp_8, fp_12, fp_16, fp_20, fp_24 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    tpr = []
    # test the performance
    for x in files:
        label = int(x[3])
        angle = x[5:8]
        if angle[0]=='0':
            rot = int(angle[0])
        elif angle[2]== '_':
            rot = int(angle[1])
        else:
            rot = int(angle[1:])
        
        IMAGE_FILE = os.path.join(test_path, x)
        input_image = caffe.io.load_image(IMAGE_FILE)
        # predict takes any number of images,
        # and formats them for the Caffe net automatically
        pred = net.predict([input_image])
        predicted = pred.argmax()
        if predicted==label:
            tp += 1
            if rot == 0:
                tp_0 += 1
            elif rot == 4:
                tp_4 += 1
            elif rot == 8:
                tp_8 += 1
            elif rot == 12:
                tp_12 += 1
            elif rot == 16:
                tp_16 += 1
            elif rot == 20:
                tp_20 += 1
            elif rot == 24:
                tp_24 += 1  
            print predicted, label, 'good'
        else:
            fp += 1
            if rot == 0:
                fp_0 += 1
            elif rot == 4:
                fp_4 += 1
            elif rot == 8:
                fp_8 += 1
            elif rot == 12:
                fp_12 += 1
            elif rot == 16:
                fp_16 += 1
            elif rot == 20:
                fp_20 += 1
            elif rot == 24:
                fp_24 += 1          
            print predicted, label, 'bad'    
    # Measure the TPR for this setup 
    tpr = [tp/(tp+fp), tp_0/(tp_0+fp_0), tp_4/(tp_4+fp_4), tp_8/(tp_8+fp_8), tp_12/(tp_12+fp_12), 
                tp_16/(tp_16+fp_16), tp_20/(tp_20+fp_20), tp_24/(tp_24+fp_24)]
   
    print ' TPR =', tpr[0], ' TPR 0 =', tpr[1], ' TPR 4 =', tpr[2], ' TPR 8 =', tpr[3]
    print ' TPR 12 =', tpr[4], ' TPR 16 =', tpr[5],' TPR 20 =', tpr[6], ' TPR 24 =', tpr[7]
    
    return tpr

In [4]:
def output_global(number_of_images, tpr):
    with open("/home/gonzalo/data/Results/Results.txt", "a") as text_file:
        text_file.write("TPR for "+ str(number_of_images) + " images: {}\n".format(tpr[0]))

In [5]:
def output_angles(number_of_images, angle_list, tpr):
    with open("/home/gonzalo/data/Results/Results_angle.txt", "a") as text_file:
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + str(angle_list[0]) + ": {}\n".format(tpr[1]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + str(angle_list[1]) + ": {}\n".format(tpr[2]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + str(angle_list[2]) + ": {}\n".format(tpr[3]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + str(angle_list[3]) + ": {}\n".format(tpr[4]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + str(angle_list[4]) + ": {}\n".format(tpr[5]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + str(angle_list[5]) + ": {}\n".format(tpr[6]))
        text_file.write("TPR for "+ str(number_of_images) + " images and rot=" + str(angle_list[6]) + ": {}\n".format(tpr[7]))

In [7]:
if __name__ == "__main__":
    
    img_number = [ 7]#, 20, 30]#, 40, 50, 100, 150, 200]
    angle_list =[0, 4, 8, 12, 16, 20, 24]
    test_path = '/home/gonzalo/data/MIT/test/test_0'
    files = os.listdir(test_path)
    
    number_of_images = 35
    train_set = 'train_0'
    epoch_set = 'top'
    model_used = 'model'+str(number_of_images)+'_'+train_set+'_'+epoch_set+'.caffemodel'
    print model_used
    
    net = load_model_net(model_used)


model35_train_0_top.caffemodel
successfully loaded classifier


In [9]:
# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

print len(files)
net.blobs['data'].reshape(len(files),        # batch size
                          3,         # 3-channel (BGR) images
                          227, 227)  # image size is 227x227

250


In [15]:
import time
#reset variables
tp, tp_0, tp_4, tp_8, tp_12, tp_16, tp_20, tp_24 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
fp, fp_0, fp_4, fp_8, fp_12, fp_16, fp_20, fp_24 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
tpr = []
inputs =[]
time_pred = 0

# test the performance outputting time consumed for face prediction of the test image. The loop
# cycles throughout the list of test images returning groundtruth label, prediction and time consumption.
for x in files[0:9]:
        label = int(x[3])
        angle = x[5:8]
        if angle[0]=='0':
            rot = int(angle[0])
        elif angle[2]== '_':
            rot = int(angle[1])
        else:
            rot = int(angle[1:])
        
        IMAGE_FILE = os.path.join(test_path, x)
        input_image = caffe.io.load_image(IMAGE_FILE)
        startTime = time.time()
        
        # Launch neural network's prediction, outputs a predictions vector with each element as a class likelihood
        pred = net.predict([input_image])
        
        # Extract the maximum value's argument as the class prediction
        predicted = pred.argmax()
        
        # Output prediction's time
        time_pred += float(time.time()-startTime)
    
print 'label= '+str(label)+'\n', 'pred= '+str(pred)+'\n', time_pred

label= 0
pred= [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
71.0676066875
